# ライブドアニュースコーパスのダウンロード

Google Driveにミラーを用意しています。オリジナルは[https://www.rondhuit.com/download.html](https://www.rondhuit.com/download.html)にあります。
論文などで引用する際は、リンク先を参照ください。

In [6]:
from google_drive_downloader import GoogleDriveDownloader as gdd

tgz_fname = "ldcc-20140209.tar.gz"

In [3]:
# Google driverからダウンロード
gdd.download_file_from_google_drive(file_id="1b-llzNQdmKIp0FYMwzGOKmXdQUNpNXC8",
                                   dest_path="./ldcc-20140209.tar.gz", unzip=False)

In [5]:
# 株式会社ロンウィットからダウンロード
#import urllib.request
#tgz_url = "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
#urllib.request.urlretrieve(tgz_url, "ldcc-20140209.tar.gz")

('ldcc-20140209.tar.gz', <http.client.HTTPMessage at 0x7ff9c2e04da0>)

# 分類用データへの加工

* tar.gzファイルから2つのジャンルを対象にする
  * text/GENRE/GENRE-#######.txt
    * 3行目がタイトルなのでこれを分類対象とする
      * 文章の前後についている【】を削除
* tsvファイルとして出力
  * フィールド構造
    1. (未使用)
    2. クラス (0/1)
    3. (未使用)
    4. テキスト

In [17]:
import tarfile
import csv
import re

target_genre = ["it-life-hack", "kaden-channel"]

zero_fnames = []
one_fnames = []
tsv_fname = "all.tsv"

brackets_tail = re.compile('【[^】]*】$')
brackets_head = re.compile('^【[^】]*】')

def remove_brackets(inp):
    output = re.sub(brackets_head, '',
                   re.sub(brackets_tail, '', inp))
    return output

def read_title(f):
    # 2行スキップ
    next(f)
    next(f)
    title = next(f) # 3行目を返す
    title = remove_brackets(title.decode('utf-8'))
    return title[:-1]

with tarfile.open(tgz_fname) as tf:
    # 対象ファイルの選定
    for ti in tf:
        # ライセンスファイルはスキップ
        if "LICENSE.txt" in ti.name:
            continue
        if target_genre[0] in ti.name and ti.name.endswith(".txt"):
            zero_fnames.append(ti.name)
            continue
        if target_genre[1] in ti.name and ti.name.endswith(".txt"):
            one_fnames.append(ti.name)
    with open(tsv_fname, "w") as wf:
        writer = csv.writer(wf, delimiter='\t')
        # ラベル 0
        for name in zero_fnames:
            f = tf.extractfile(name)
            title = read_title(f)
            row = [target_genre[0], 0, '', title]
            writer.writerow(row)
        # ラベル 1
        for name in one_fnames:
            f = tf.extractfile(name)
            title = read_title(f)
            row = [target_genre[1], 1, '', title]
            writer.writerow(row)


# データのシャッフルと分割

* train/dev/test : 8:1:1 で分割

In [9]:
import random

random.seed(100)
with open("all.tsv", 'r') as f, open("rand-all.tsv", "w") as wf:
    lines = f.readlines()
    random.shuffle(lines)
    for line in lines:
        wf.write(line)

In [10]:
random.seed(101)

train_fname, dev_fname, test_fname = ["train.tsv", "dev.tsv", "test.tsv"]

with open("rand-all.tsv") as f, open(train_fname, "w") as tf, open(dev_fname, "w") as df, open(test_fname, "w") as ef:
    ef.write("class\tsentence\n")
    for line in f:
        v = random.randint(0, 9)
        if v == 8:
            df.write(line)
        elif v == 9:
            row = line.split('\t')
            ef.write("\t".join([row[1], row[3]]))
        else:
            tf.write(line)

# BERTのセットアップ

* git repoの用意
* Multilingualモデルの用意

In [22]:
import subprocess
import os

if not os.path.exists("bert"):
    subprocess.call("git clone https://github.com/google-research/bert".split())

In [23]:
import urllib.request
ml_url = "https://storage.googleapis.com/bert_models/2018_11_03/multilingual_L-12_H-768_A-12.zip"
urllib.request.urlretrieve(ml_url, "multilingual_L-12_H-768_A-12.zip")

('multilingual_L-12_H-768_A-12.zip',
 <http.client.HTTPMessage at 0x7ff9c05f6da0>)

In [24]:
! unzip multilingual_L-12_H-768_A-12.zip

Archive:  multilingual_L-12_H-768_A-12.zip
   creating: multilingual_L-12_H-768_A-12/
  inflating: multilingual_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multilingual_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multilingual_L-12_H-768_A-12/vocab.txt  
  inflating: multilingual_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: multilingual_L-12_H-768_A-12/bert_config.json  
